In [1]:
!python -m pip install findspark

In [1]:
import findspark

findspark.init()

In [4]:
import pyspark

from pyspark.sql import SparkSession

sc = SparkSession.builder.getOrCreate()

df = sc.sql("select 'spark' as hello ")

df.show()

+-----+
|hello|
+-----+
|spark|
+-----+



In [6]:
import pandas as pd
df = pd.read_csv("https://github.com/manjiler/interview_for_datascience/raw/master/interview_df.csv")

In [64]:
# Pandas to Spark
df_sp = sc.createDataFrame(df)

In [65]:
df_sp.printSchema()

root
 |-- s1: double (nullable = true)
 |-- s2: double (nullable = true)
 |-- s3: double (nullable = true)
 |-- s4: double (nullable = true)
 |-- s5: double (nullable = true)
 |-- s6: double (nullable = true)
 |-- s7: double (nullable = true)
 |-- output: double (nullable = true)
 |-- datetimeindex: string (nullable = true)
 |-- timestamp: long (nullable = true)



In [66]:
from pyspark.sql.functions import concat_ws,col
from pyspark.sql.functions import *
df_sp = df_sp.withColumn("unix_timestamp", concat_ws(".",from_unixtime((col("timestamp")/1000),"yyyy-MM-dd HH:mm:ss"),substring(col("timestamp"),-3,3)))

In [69]:
df_sp = df_sp.withColumn('dt',df_sp.unix_timestamp.astype('Timestamp').cast("long"))

In [14]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

In [61]:
w

In [56]:
w = Window.partitionBy('s1').orderBy('timestamp').rangeBetween(-60*5,0)
df_sp = df_sp.withColumn('occurrences_in_5_min',F.sum('s1').over(w))
df_sp.select('timestamp','datetimeindex','unix_timestamp','s1','occurrences_in_5_min').show()


+-------------+-------------------+--------------------+--------------------+--------------------+
|    timestamp|      datetimeindex|      unix_timestamp|                  s1|occurrences_in_5_min|
+-------------+-------------------+--------------------+--------------------+--------------------+
|1557520200000|2019-05-10 20:30:00|2019-05-11 02:00:...|  -1.122909152988955|  -1.122909152988955|
|1557122400000|2019-05-06 06:00:00|2019-05-06 11:30:...| -1.1046194445047708| -1.1046194445047708|
|1556865000000|2019-05-03 06:30:00|2019-05-03 12:00:...| -1.0055371008236342| -1.0055371008236342|
|1557341100000|2019-05-08 18:45:00|2019-05-09 00:15:...| -0.9461396928097158| -0.9461396928097158|
|1558050900000|2019-05-16 23:55:00|2019-05-17 05:25:...| -0.9399509244655836| -0.9399509244655836|
|1557050400000|2019-05-05 10:00:00|2019-05-05 15:30:...| -0.8362756020717549| -0.8362756020717549|
|1557126600000|2019-05-06 07:10:00|2019-05-06 12:40:...| -0.7425705773741609| -0.7425705773741609|
|155844990

In [74]:
w = Window.partitionBy('dt').orderBy('dt').rangeBetween(-60*60,0)
df_sp = df_sp.withColumn('occurrences_in_5_min',F.avg('s1').over(w))
df_sp.select('timestamp','datetimeindex','s1','unix_timestamp','occurrences_in_5_min').orderBy('dt').show()

+-------------+-------------------+--------------------+--------------------+--------------------+
|    timestamp|      datetimeindex|                  s1|      unix_timestamp|occurrences_in_5_min|
+-------------+-------------------+--------------------+--------------------+--------------------+
|1556668800000|2019-05-01 00:00:00| -1.3964757857491117|2019-05-01 05:30:...| -1.3964757857491117|
|1556669100000|2019-05-01 00:05:00| 0.11270720451231787|2019-05-01 05:35:...| 0.11270720451231787|
|1556669400000|2019-05-01 00:10:00|  0.9237635795544692|2019-05-01 05:40:...|  0.9237635795544692|
|1556669700000|2019-05-01 00:15:00|  1.2620335463316603|2019-05-01 05:45:...|  1.2620335463316603|
|1556670000000|2019-05-01 00:20:00|0.032973209435392684|2019-05-01 05:50:...|0.032973209435392684|
|1556670300000|2019-05-01 00:25:00|  0.3453403091266413|2019-05-01 05:55:...|  0.3453403091266413|
|1556670600000|2019-05-01 00:30:00|  1.2345760686504736|2019-05-01 06:00:...|  1.2345760686504736|
|155667090

In [80]:
df_sp = df_sp.withColumn('sys', lit('sys'))
df_sp.show()

+--------------------+--------------------+-------------------+--------------------+--------------------+------------------+--------------------+-------------------+-------------------+-------------+--------------------+--------------------+----------+---+
|                  s1|                  s2|                 s3|                  s4|                  s5|                s6|                  s7|             output|      datetimeindex|    timestamp|      unix_timestamp|occurrences_in_5_min|        dt|sys|
+--------------------+--------------------+-------------------+--------------------+--------------------+------------------+--------------------+-------------------+-------------------+-------------+--------------------+--------------------+----------+---+
|-0.25583754618518145|  -1.565651965697432| 0.2338399216695104|-0.04622016912774...|  0.8543718114365333|1.6027497146505216| -0.1695719851837311| -235.5885286963713|2019-05-01 22:25:00|1556749500000|2019-05-02 03:55:...|-0.255837

In [82]:
from pyspark.sql.window import Window
windowSpec = Window().partitionBy(['sys']).orderBy('dt').rowsBetween(Window.unboundedPreceding,Window.currentRow)
df_sp = df_sp.withColumn("cumulative_confirmed",F.sum("s1").over(windowSpec))


In [83]:
df_sp.select('timestamp','datetimeindex','unix_timestamp','s1','cumulative_confirmed').show()

+-------------+-------------------+--------------------+--------------------+--------------------+
|    timestamp|      datetimeindex|      unix_timestamp|                  s1|cumulative_confirmed|
+-------------+-------------------+--------------------+--------------------+--------------------+
|1556668800000|2019-05-01 00:00:00|2019-05-01 05:30:...| -1.3964757857491117| -1.3964757857491117|
|1556669100000|2019-05-01 00:05:00|2019-05-01 05:35:...| 0.11270720451231787| -1.2837685812367938|
|1556669400000|2019-05-01 00:10:00|2019-05-01 05:40:...|  0.9237635795544692|-0.36000500168232463|
|1556669700000|2019-05-01 00:15:00|2019-05-01 05:45:...|  1.2620335463316603|  0.9020285446493357|
|1556670000000|2019-05-01 00:20:00|2019-05-01 05:50:...|0.032973209435392684|  0.9350017540847284|
|1556670300000|2019-05-01 00:25:00|2019-05-01 05:55:...|  0.3453403091266413|  1.2803420632113698|
|1556670600000|2019-05-01 00:30:00|2019-05-01 06:00:...|  1.2345760686504736|  2.5149181318618434|
|155667090

In [108]:
from pyspark.sql.window import Window
windowSpec = Window().partitionBy(['sys']).orderBy('dt').rangeBetween(-60*5 ,0)
df_sp = df_sp.withColumn("roll_7_confirmed",F.sum("s1").over(windowSpec))

In [109]:
df_sp.select('timestamp','datetimeindex','unix_timestamp','s1','roll_7_confirmed').orderBy('unix_timestamp').show()

+-------------+-------------------+--------------------+--------------------+-------------------+
|    timestamp|      datetimeindex|      unix_timestamp|                  s1|   roll_7_confirmed|
+-------------+-------------------+--------------------+--------------------+-------------------+
|1556668800000|2019-05-01 00:00:00|2019-05-01 05:30:...| -1.3964757857491117|-1.3964757857491117|
|1556669100000|2019-05-01 00:05:00|2019-05-01 05:35:...| 0.11270720451231787|-1.2837685812367938|
|1556669400000|2019-05-01 00:10:00|2019-05-01 05:40:...|  0.9237635795544692|  1.036470784066787|
|1556669700000|2019-05-01 00:15:00|2019-05-01 05:45:...|  1.2620335463316603| 2.1857971258861295|
|1556670000000|2019-05-01 00:20:00|2019-05-01 05:50:...|0.032973209435392684|  1.295006755767053|
|1556670300000|2019-05-01 00:25:00|2019-05-01 05:55:...|  0.3453403091266413|  0.378313518562034|
|1556670600000|2019-05-01 00:30:00|2019-05-01 06:00:...|  1.2345760686504736|  1.579916377777115|
|1556670900000|2019-

In [111]:
from pyspark.sql.window import Window
windowSpec = Window().partitionBy(['sys']).orderBy('dt').rangeBetween(-60*5 ,-1)
df_sp = df_sp.withColumn("roll_7_confirmed",F.sum("s1").over(windowSpec))
df_sp.select('timestamp','datetimeindex','unix_timestamp','s1','roll_7_confirmed').orderBy('unix_timestamp').show()

+-------------+-------------------+--------------------+--------------------+--------------------+
|    timestamp|      datetimeindex|      unix_timestamp|                  s1|    roll_7_confirmed|
+-------------+-------------------+--------------------+--------------------+--------------------+
|1556668800000|2019-05-01 00:00:00|2019-05-01 05:30:...| -1.3964757857491117|                null|
|1556669100000|2019-05-01 00:05:00|2019-05-01 05:35:...| 0.11270720451231787| -1.3964757857491117|
|1556669400000|2019-05-01 00:10:00|2019-05-01 05:40:...|  0.9237635795544692| 0.11270720451231787|
|1556669700000|2019-05-01 00:15:00|2019-05-01 05:45:...|  1.2620335463316603|  0.9237635795544692|
|1556670000000|2019-05-01 00:20:00|2019-05-01 05:50:...|0.032973209435392684|  1.2620335463316603|
|1556670300000|2019-05-01 00:25:00|2019-05-01 05:55:...|  0.3453403091266413|0.032973209435392684|
|1556670600000|2019-05-01 00:30:00|2019-05-01 06:00:...|  1.2345760686504736|  0.3453403091266413|
|155667090

In [114]:
from pyspark.sql.window import Window
windowSpec = Window().partitionBy(['sys']).orderBy('dt').rangeBetween(-60*20 ,-1)
df_sp = df_sp.withColumn("roll_7_confirmed",F.sum("s1").over(windowSpec))
df_sp.select('timestamp','datetimeindex','unix_timestamp','s1','roll_7_confirmed').orderBy('unix_timestamp').show()

+-------------+-------------------+--------------------+--------------------+--------------------+
|    timestamp|      datetimeindex|      unix_timestamp|                  s1|    roll_7_confirmed|
+-------------+-------------------+--------------------+--------------------+--------------------+
|1556668800000|2019-05-01 00:00:00|2019-05-01 05:30:...| -1.3964757857491117|                null|
|1556669100000|2019-05-01 00:05:00|2019-05-01 05:35:...| 0.11270720451231787| -1.3964757857491117|
|1556669400000|2019-05-01 00:10:00|2019-05-01 05:40:...|  0.9237635795544692| -1.2837685812367938|
|1556669700000|2019-05-01 00:15:00|2019-05-01 05:45:...|  1.2620335463316603|-0.36000500168232463|
|1556670000000|2019-05-01 00:20:00|2019-05-01 05:50:...|0.032973209435392684|  0.9020285446493357|
|1556670300000|2019-05-01 00:25:00|2019-05-01 05:55:...|  0.3453403091266413|  2.3314775398338403|
|1556670600000|2019-05-01 00:30:00|2019-05-01 06:00:...|  1.2345760686504736|  2.5641106444481636|
|155667090